In [1]:
import datetime

import numpy as np
import pandas as pd

import vectorbt as vbt
import talib

from numba import njit

In [2]:
end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(days=7)

interval = "1m"

In [3]:
symbols = [
    "BTC-USD",
    "ETH-USD",
    "ETH-BTC",
]

In [4]:
# price = vbt.YFData.download(
#     symbols = symbols,
#     missing_index='drop',
#     interval=interval,
#     start = start_time,
#     end = end_time
# ).get('Close')

# price.to_pickle('price.pkl')

In [5]:
price = pd.read_pickle('price.pkl')
price

symbol,BTC-USD,ETH-USD,ETH-BTC
Datetime,,,
2023-05-22 22:39:00+00:00,26888.099609,1820.341431,0.067700
2023-05-22 22:40:00+00:00,26888.451172,1820.353271,0.067700
2023-05-22 22:41:00+00:00,26887.414062,1820.118652,0.067703
2023-05-22 22:42:00+00:00,26888.544922,1820.242188,0.067691
2023-05-22 22:43:00+00:00,26892.275391,1820.446655,0.067686
...,...,...,...
2023-05-29 22:30:00+00:00,27590.566406,1888.025513,0.068433
2023-05-29 22:31:00+00:00,27587.107422,1887.651733,0.068439
2023-05-29 22:32:00+00:00,27586.919922,1887.667725,0.068426


In [6]:
def custom_indicator_slow(close, rsi_window = 14, entry_rsi = 30, exit_rsi = 70):
    # print dimensions of close
    # print(close.shape)
    rsi = vbt.RSI.run(close, rsi_window).rsi
    trend = np.where(rsi > exit_rsi, -1, 0)
    trend = np.where(rsi < entry_rsi, 1, trend)
    return trend

In [7]:
@njit
def produce_signal(rsi, entry_rsi, exit_rsi):
    trend = np.where(rsi > exit_rsi, -1, 0)
    trend = np.where((rsi < entry_rsi), 1, trend)
    return trend

# wrapper for talib RSI, because the dimensions of the input are different
RSI = vbt.IndicatorFactory.from_talib('RSI')

# this is the same as the above function but using talib
def custom_indicator_fast(close, rsi_window = 14, entry_rsi = 30, exit_rsi = 70):
    rsi = RSI.run(close, rsi_window).real.to_numpy()
    return produce_signal(rsi, entry_rsi, exit_rsi)

In [8]:
# custom_indicator_slow(price)

In [9]:
# custom_indicator_fast(price)

In [10]:
indicator = vbt.IndicatorFactory(
    class_name = 'Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window', 'entry_rsi', 'exit_rsi'],
    output_names=['value']
).from_apply_func(
    custom_indicator_fast,
    rsi_window=14,
    entry_rsi=30,
    exit_rsi=70,
)

In [11]:
res = indicator.run(price)
# res

In [12]:
res = indicator.run(
    price,
    rsi_window=np.arange(start=10, stop=60, step=1, dtype=int),
    entry_rsi=np.arange(start=5, stop=45, step=5, dtype=float),
    exit_rsi=np.arange(start=55, stop=95, step=5, dtype=float),
    param_product=True
)


In [13]:
entries = res.value == 1
exits = res.value == -1
pf = vbt.Portfolio.from_signals(price, entries, exits, fees=0.001, freq=interval)

In [14]:
returns = pf.total_return()

print(f'Best params: {returns.idxmax()} with return {returns.max()}')
print(f'Worst params: {returns.idxmin()} with return {returns.min()}')

Best params: (33, 15.0, 85.0, 'ETH-USD') with return 0.0583706376991583
Worst params: (10, 40.0, 55.0, 'BTC-USD') with return -0.38552528848817647


In [15]:
# pf.stats()

In [16]:
returns = pf.total_return()

best_params = returns.idxmax()
worst_params = returns.idxmin()

print(f'Best params: {best_params} with return {returns.max()}')
print(f'Worst params: {worst_params} with return {returns.min()}')

Best params: (33, 15.0, 85.0, 'ETH-USD') with return 0.0583706376991583
Worst params: (10, 40.0, 55.0, 'BTC-USD') with return -0.38552528848817647


In [17]:
returns

comb_rsi_window  comb_entry_rsi  comb_exit_rsi  symbol 
10               5.0             55.0           BTC-USD   -0.002207
                                                ETH-USD    0.001362
                                                ETH-BTC    0.000000
                                 60.0           BTC-USD    0.000625
                                                ETH-USD    0.002837
                                                             ...   
59               40.0            85.0           ETH-USD    0.039231
                                                ETH-BTC    0.012142
                                 90.0           BTC-USD    0.029027
                                                ETH-USD    0.039231
                                                ETH-BTC    0.012142
Name: total_return, Length: 9600, dtype: float64

In [18]:
fig = returns.vbt.volume(
    x_level='comb_rsi_window',
    y_level='comb_entry_rsi', 
    z_level='comb_exit_rsi',
    slider_level='symbol',
    trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='%')))
fig.show()